# American Visitors
### Udacity Data Engineering Capstone Project  

#### Project Summary
In this project, where the main goal was to combine what I've learned
throughout the program I decided to use datasets included in the Project Workspace. 
I decided to concentrate on two datasets:
- **I94 Immigration Data**: This data comes from the US National Tourism and Trade Office.
[Data source](https://travel.trade.gov/research/reports/i94/historical/2016.html)  
  I wanted to deal with the processing of a larger set of data provided by Udacity
  in `../../data/18-83510-I94-Data-2016/` location of Project Workspace.
- **Airport Code Table** - simple table of airport codes and corresponding cities.
  Data cames from [this location](https://datahub.io/core/airport-codes#data)
  and available in `airport-codes_csv.csv` file.  
  
additionaly I touch a little **U.S. City Demographic Data** but in case of possible way
of extending information in target DWH.   

**Project scope**: The main dataset includes data on immigration to the United State. Within this project I transform dataset from SAS format using Spark process and then in Airflow I transforming partitioned and cleaned data from AWS S3 to Redshift Data Warehouse (DWH) for further Business Intelligence analysis.

**Use Cases**:  
Final solution in AWS Redshift DWH shall allow to answer for example questions:

- What are most visited US states for visitors?
- How many people visit US in 2016 each month on a business visa?  
- How many days in average each visitor stayed in US in each month?  

**Project** is devided on two *subprojects*:  

- **Jupyter Notebook** where I follow:
  - (Step 1) - Identifing and gathering the data  
  - (Step 2) - Exploring and Assessing the Data  
   In this subproject I use Spark inside Udacity Project Workspace to process I94imigration data
    and finally store results in AWS S3 bucket for further processing using Airflow.    
- **Airflow** where:
  - (Step 3) - Data Model is define and implemented 
  - (Step 4) - ETL process is implemented   
    In this subproject AWS Redshift and Airflow is used.  
    More description provided in **Airflow** section below in `README.md` file

In [42]:
# Do all imports and installs here
import pandas as pd
import numpy as np
import os
import configparser
import io

config = configparser.ConfigParser()

#Normally this file should be in ~/.aws/credentials
config.read_file(open('dwh.cfg'))

os.environ["AWS_ACCESS_KEY_ID"]= config['AWS']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"]= config['AWS']['AWS_SECRET_ACCESS_KEY']

import boto3
from botocore.exceptions import ClientError
import json

def write_pandas_to_json_on_s3(df, file_dest_name):
    """
    Convert pandas df to json with orient='record' and save data to S3 bucket
    """
    s3 = boto3.resource('s3')
    s3_bucket = s3.Bucket(config['S3']['PROJECT_BUCKET'])
    
    json_buffer = io.StringIO()
    #df.to_json(json_buffer,orient='records')
    # Becose of COPY {} FROM JSON in Redshift we nead to prepare one file with multiple JSON records
    # array is not supported
    json_buffer.write('\n'.join([json.dumps(row) for row in df.to_dict('records')]))

    try:
        s3_bucket.put_object(Key=file_dest_name, Body=json_buffer.getvalue())
    except Exception as e:
        print(e)

        
def upload_files_to_s3(local_path):
    """
    Uploads all files from `local_path` location to S3 bucket
    """
    # normalize local_path name
    local_path = os.path.relpath(local_path)
    
    # S3 bucket obj def
    s3 = boto3.resource('s3')
    s3_bucket = s3.Bucket(config['S3']['PROJECT_BUCKET'])

    # path walk
    for subdir, dirs, files in os.walk(local_path):
        files = [f for f in files if not f[0] == '.']
        for file in files:
            full_path = os.path.join(subdir, file)
            # print(full_path)
            with open(full_path, 'rb') as data:                
                s3_bucket.put_object(Key=full_path, Body=data)

## Step 1: Scope the Project and Gather Data

### Scope 
I have identified following important data mapping for further processing.  

![airflow_subdag](./img/file_mapping.png)

---
### Describe and Gather Data 
I have define followind steps to do:
1. Extract and prepare lookup data witch will be used as Dimensions (from I94_SAS_Labels_Descriptions.SAS file)
2. I94 Port data enrichment with data from **Airport Code Table** `airport-codes_csv.csv` including data cleaning and merging
3. Preparing data to import by Redshift in JSON format and storing data in AWS S3
4. Gethering I94 Imigration data from `../../data/18-83510-I94-Data-2016/` location using PySpark:  
    i. cleaning the data  (part of Step 2)  
    ii. partitioning by year and mounth and storung parquet format in `sas_data_full` location  
5. Send sas_data_full to do S3 (for further analysis using Airflow)

**NOTE:** Please check `dwh.cfg` file for proper configure AWS credentials and S3 bucket names

## Step 1.1. Extract and prepare lookup data from I94_SAS_Labels_Descriptions.SAS
I decided to use sed.  
Result are stored in `lookup_data` folder

In [2]:
!echo "location_id;location_name"> lookup_data/_i94cntyl.csv
!sed -n "/value i94cnty/,/;/s/\ \+\([0-9]*\)\ \+= \+'\(.*\)'.\?\+/\1;\2/p" I94_SAS_Labels_Descriptions.SAS >> lookup_data/_i94cntyl.csv

!echo "mode_id;mode_name"> lookup_data/_i94model.csv
!sed -n "/i94model/,/;/s/ \+\([0-9]*\).\?\+=.\?\+'\(.*\)'.\?\+/\1;\2/p" I94_SAS_Labels_Descriptions.SAS >> lookup_data/_i94model.csv

!echo "port_code;port_name"> lookup_data/_i94prtl.csv
!sed -n "/value \$i94prtl/,/;/s/\ \+'\(.*\)'.\?\+=.\?\+'\(.*\)'/\1;\2/p" I94_SAS_Labels_Descriptions.SAS >> lookup_data/_i94prtl.csv

!echo "address_code;address_name"> lookup_data/_i94addrl.csv
!sed -n "/value i94addrl/,/;/s/\t\+'\(.*\)'.\?\+=.\?\+'\(.*\)'.\?\+/\1;\2/p" I94_SAS_Labels_Descriptions.SAS >> lookup_data/_i94addrl.csv

!echo "visa_id;visa_category"> lookup_data/_i94visa.csv
!sed -n "/^\/\* I94VISA/,/\*\//s/.\?\+\([0-9]\).\?\+= \?\+\(.*\)/\1;\2/p" I94_SAS_Labels_Descriptions.SAS >> lookup_data/_i94visa.csv

## Step 1.2  - I94Port data enrichment with data from Airport Code Table 

In [3]:
# Read in the data here
df_airport = pd.read_csv('airport-codes_csv.csv')

In [4]:
# DATA check during DATA EXPLORATION
# df_airport.head()
# df_airport.columns.tolist()

In [5]:
# extract 'state' from iso_region
df_airport['state'] = df_airport['iso_region'].str.split('-',expand=True)[1]

In [6]:
# split coordinates to Lot,Lat
df_airport[['lon','lat']] = df_airport['coordinates'].str.split(',',expand=True)
del df_airport['coordinates']
# df_airport.head()

In [7]:
# get port data from SAS lables Description
df_i94prt = pd.read_csv('lookup_data/_i94prtl.csv',sep=";")

In [8]:
# split port_name to name and state
df_i94prt[['port_name','state']] = df_i94prt['port_name'].str.split(",",expand=True)[[0,1]]
# remove whitespaces
df_i94prt['state'] = df_i94prt['state'].str.strip()

In [9]:
def clean_state_string(s):
    "Simple state cleaning opearation base on manual data check"
    if s is None:
        return None
    if "(" in s:
        #print(s)
        return s[:2]
    if "#" in s:
        #print(s)
        return s.split(" ")[0]
    return s

In [10]:
# apply cleaning
df_i94prt['state'] = df_i94prt['state'].apply(clean_state_string) 

In [11]:
# Leave only 2-letters data in 'state column, rest in 'info
df_i94prt['info'] = df_i94prt[df_i94prt['state'].str.len() > 2]['state']
df_i94prt['state'] = df_i94prt[df_i94prt['state'].str.len() <=2]['state']

In [12]:
#check
# df_i94prt.groupby('state').count()
# df_i94prt.groupby('info').count()

# check airport types
# df_airport['type'].unique().tolist()

In [13]:
# remove closed airports
df_airport=df_airport[df_airport['type'] !='closed']

In [14]:
# define collumns to select
airport_collumns = [
    'port_code',  # new column from SAS combine base on 'gps_code', 'iata_code', 'local_code'
    'ident',    
    'type',
    'name',
    'elevation_ft',
    'continent',
    'iso_country',
    'state',
    'municipality',
    'lon',
    'lat']

In [15]:
# iterate on 'gps_code', 'iata_code', 'local_code' to find airports from df_i94prt
df_airport_final = None
# for c in ['gps_code', 'iata_code', 'local_code']: # finaly I have decided to use only gps_code and iata_code
for c in ['gps_code', 'iata_code']:
    _dfTmp = df_airport[df_airport[c].isin(df_i94prt['port_code'])].copy()
    _dfTmp['port_code'] = _dfTmp[c]
    _dfTmp = _dfTmp[airport_collumns]
    
    if df_airport_final is None:
        df_airport_final = _dfTmp
    else:
        df_airport_final = df_airport_final.append(_dfTmp)

In [16]:
# finaly drop duplicates
df_airport_final.drop_duplicates(inplace=True)

In [17]:
# LEFT jonin all 'valid' to i94 SAS port data (base on port and state validation)
df_i94prt_final = pd.merge(df_i94prt, df_airport_final, on=['port_code','state'], how='left')

In [18]:
# save final i94port data
df_i94prt_final.to_csv('lookup_data/_i94prtl.csv',sep=';',index=False)

In [19]:
df_i94prt_final.head()

,port_code,port_name,state,info,ident,type,name,elevation_ft,continent,iso_country,municipality,lon,lat
0,ALC,ALCAN,AK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ANC,ANCHORAGE,AK,NaN,PANC,large_airport,Ted Stevens Anchorage International Airport,152.0,NaN,US,Anchorage,-149.99600219726562,61.174400329589844
2,BAR,BAKER AAF - BAKER ISLAND,AK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DAC,DALTONS CACHE,AK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PIZ,DEW STATION PT LAY DEW,AK,NaN,PPIZ,medium_airport,Point Lay LRRS Airport,22.0,NaN,US,Point Lay,-163.0050049,69.73290253


## Step 1.3. Preparing data to import by Redshift in JSON format and storing data in AWS S3  
NOTE: bucket definition is configured in dwh.cfg in [S3] section

In [20]:
# read all extracted data from I94_SAS_Labels_Descriptions.SAS and write to S3 in JSON format
lookup_data = [
    '_i94cntyl',
    '_i94prtl',
    '_i94model',
    '_i94addrl',
    '_i94visa'
]

for n in lookup_data:
    df = pd.read_csv(f'lookup_data//{n}.csv',sep=';')
    df = df.replace({np.nan:None})
    # write to S3 bucket in JSON format supported by AWS Redshift
    write_pandas_to_json_on_s3(df,f'lookup_data/{n}.json')
    print(f'Processing {n} - done')

Processing _i94cntyl - done
Processing _i94prtl - done
Processing _i94model - done
Processing _i94addrl - done
Processing _i94visa - done


NOTE: This data will be used to fill dimension tables i target AMVisitors DB

## Step 1.4. Exploration of sample data set `immigration_data_sample.csv`
Goal: Identification which collumns are usefull and what need to be remove in target PySpark processing
NOTE: this section is only for information use to informe about the process of 'DATA EXPLORATION'

During this part most of ## Step 2: Explore and Assess the Data were done.
Results were implemented in final PySpark ETL process

In [21]:
df_imigrations = pd.read_csv('immigration_data_sample.csv')

In [22]:
# df_imigrations.columns.tolist()

In [23]:
# df_imigrations['count'].describe()
# count CONCLUSTION: no usefull information, TO_DEL

##### analysis od data market in I94_SAS_Labels_Descriptions.SAS as "CIC does not use"

In [24]:
# df_imigrations[['count',
#                 'dtadfile', #CIC does not use
#                 'visapost', #CIC does not use
#                 'occup',    #CIC does not use
#                 'entdepa',  #CIC does not use
#                 'entdepd',  #CIC does not use
#                 'entdepu',  #CIC does not use
#                 'matflag',  #CIC does not use
#                 'biryear',  #???
#                 'dtaddto',  #CIC does not use
#                 'insnum', #????
#                 'admnum', #????
#             ]].info()

In [25]:
# df_imigrations[['count',
#                 'dtadfile', #CIC does not use
#                 'visapost', #CIC does not use
#                 'occup',    #CIC does not use
#                 'entdepa',  #CIC does not use
#                 'entdepd',  #CIC does not use
#                 'entdepu',  #CIC does not use
#                 'matflag',  #CIC does not use
#                 'biryear',  #???
#                 'dtaddto',  #CIC does not use
#                 'insnum', #????
#                 'admnum', #????
#             ]].head()

In [26]:
# df_imigrations[['count',
#                 'dtadfile', #CIC does not use
#                 'visapost', #CIC does not use
#                 'occup',    #CIC does not use
#                 'entdepa',  #CIC does not use
#                 'entdepd',  #CIC does not use
#                 'entdepu',  #CIC does not use
#                 'matflag',  #CIC does not use
#                 'biryear',  #???
#                 'dtaddto',  #CIC does not use
#                 'insnum', #????
#                 'admnum', #????
#             ]].describe()

In [27]:
%%html
<style>
table {float:left}
</style>

|field|description|
|-----|-----------|
|cicid|---|
|i94yr|4 digit year|
|i94mon|Numeric month|
|i94cit| ref. to i94cntyl codes (file: i94cntyl.json)|
|i94res| ref. to i94cntyl codes (file: i94cntyl.json)|
|i94port| ref. to i94prtl codes (file: i94prtl.json)|
|arrdate| is the Arrival Date in the USA|
|i94mode| ref. to i94model (file: i94model.json). There are missing values as well as not reported (9)|
|i94addr| ref. to i94addr codes (file: i94addr.json)|
|depdate| is the Departure Date from the USA.|
|i94bir|Age of Respondent in Years|
|i94visa| ref. to i94visa Visa codes (file: i94visa class)|
|count|---|
|dtadfile|Character Date Field - Date added to I-94 Files - CIC does not use|
|visapost|Department of State where where Visa was issued - CIC does not use|
|occup|Occupation that will be performed in U.S. - CIC does not use (a few records|
|entdepa|Arrival Flag - admitted or paroled into the U.S. - CIC does not use|
|entdepd|Departure Flag - Departed, lost I-94 or is deceased - CIC does not use|
|entdepu|Update Flag - Either apprehended, overstayed, adjusted to perm residence - CIC does not use|
|matflag|Match flag - Match of arrival and departure records|
|biryear|4 digit year of birth|
|dtaddto|Character Date Field - Date to which admitted to U.S. (allowed to stay until) - CIC does not use|
|gender|Non-immigrant sex|
|insnum|INS number|
|airline|Airline used to arrive in U.S.|
|admnum|Admission Number|
|fltno|Flight number of Airline used to arrive in U.S.|
|visatype|Class of admission legally admitting the non-immigrant to temporarily stay in U.S.|

## Step 1.5. Gethering I94 Imigration data from `../../data/18-83510-I94-Data-2016/` location using PySpark
**NOTE**: main idea is to process I94 data in SAS format, clean and store to sas_data_full folder in parquet format

### PySpark python modules import

In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
import pyspark.sql.functions as F

---

### Function definition

In [29]:
def create_spark_session():
    """
    SparkSession getOrCreate with support of sas7bdat data read.
    """
    spark = SparkSession.builder.\
    config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
    .enableHiveSupport().getOrCreate()
    return spark

In [30]:
import datetime
def convert_sasdate_to_yyyymmdd(sasdate):
    if sasdate !=-1:
        epoch = datetime.datetime(1960, 1, 1)
        return (epoch + datetime.timedelta(days=sasdate)).strftime('%Y%m%d')
    else:
        epoch = datetime.datetime(1900, 1, 2)
        return (epoch + datetime.timedelta(days=sasdate)).strftime('%Y%m%d')

In [31]:
def process_sas_data_and_save_to_parquet(df, destination_path):
    """
    Data Process transformation and cleaning on spark DataFrame.
    Result are sawed in parquet format in provided location.
    Params:
    
    """
    # Int columns - DATA CLEAN / Normalization
    null_int_cols = {'cicid': -1, 
                     'i94yr': -1,
                     'i94mon': -1,
                     'i94cit': 999, #'INVALID: UNKNOWN' from I94_SAS_Labels_Descriptions.SAS
                     'i94res': 239, #'INVALID: UNKNOWN COUNTRY' from I94_SAS_Labels_Descriptions.SAS
                     'i94mode': 9,  #'Not reported' from  from I94_SAS_Labels_Descriptions.SAS
                     'arrdate': -1,
                     'depdate': -1,
                     'i94bir': -1,
                     'i94visa': -1,
                     'biryear': -1}

    for k in null_int_cols:
        df = df.withColumn(k, F.when((F.col(k).isNull()), null_int_cols[k]).otherwise(F.col(k).cast("int")))

    # String columns DATA Clean / Normalization
    null_str_cols = {'i94port': 'XXX', # NOT REPORTED/UNKNOWN from I94_SAS_Labels_Descriptions.SAS
                     'i94addr': '99',  #'All Other Codes' from I94_SAS_Labels_Descriptions.SAS
                     'visatype': 'unknown',
                     'gender': 'U',    # one letter (M-male,F-female,U-unknown)
                     'insnum':'unknown',
                     'airline': 'unknown',
                     'fltno': 'unknown',
                     'occup': 'unknown'}

    for k in null_str_cols:
        df = df.withColumn(k, F.when((F.col(k).isNull()), null_str_cols[k]).otherwise(F.col(k)))

    udf_sasdate_convert = udf(lambda x: convert_sasdate_to_yyyymmdd(x))
    df = df.withColumn('arrdate', udf_sasdate_convert(df['arrdate']))
    df = df.withColumn('depdate', udf_sasdate_convert(df['depdate']))

    # drop unused collumns
    # in file I94_SAS_Labels_Descriptions.SAS desc. "CIC does not use"
    drop_cols = [
                    'count',
                    'dtadfile', #CIC does not use
                    'visapost', #CIC does not use
                    'entdepa',  #CIC does not use
                    'entdepd',  #CIC does not use
                    'entdepu',  #CIC does not use
                    'matflag',  #CIC does not use
                    'dtaddto',  #CIC does not use
                    'validres',    # appears in Month=6    
                    'delete_days', # appears in Month=6
                    'delete_mexl', # appears in Month=6
                    'delete_dup',  # appears in Month=6
                    'delete_visa', # appears in Month=6
                    'delete_recdup'# appears in Month=6
               ]
    df = df.drop(*drop_cols)

    # admnum - could be usefull so conversion to long
    df = df.withColumn("admnum", df['admnum'].cast('Decimal(11,0)').cast("long"))

    # Columns for partitioning purpose
    df = df.withColumn("year", df["i94yr"])
    df = df.withColumn("month", df["i94mon"])

    df.write.parquet(destination_path, partitionBy=["year", "month"], mode='append')

In [32]:
def get_path_sas_files(path_sas_folder):
    """
    Get full list of sas7bat files in path_sas_folder location
    """
    os.listdir(path_sas_folder)
    files_data_sas = []
    path_sas_files = [os.path.join(path_sas_folder, file_path) for file_path in os.listdir(path_sas_folder) if
                      file_path[-8:] == 'sas7bdat']
    return path_sas_files

### Spark data process RUN

In [33]:
spark = create_spark_session()

In [34]:
# get array of all sas data files
sas_files = get_path_sas_files('../../data/18-83510-I94-Data-2016/')

In [35]:
# if we want to process one file
# sas_files = sas_files[4:5]

In [37]:
# for f in sas_files:
#     print(f)

In [38]:
%%time
for f in sas_files:
    df_spark =spark.read.format('com.github.saurfang.sas.spark').load(f)
    process_sas_data_and_save_to_parquet(df_spark,'sas_data_full')

CPU times: user 81.7 ms, sys: 27.9 ms, total: 110 ms
Wall time: 2min 33s


---

### data check

In [39]:
# df_spark =spark.read.parquet('sas_data_full/year=2016/month=4/')
df_spark =spark.read.parquet('sas_data_full/')

In [41]:
df_spark.count()

3574989

In [40]:
len(df_spark.columns)

22

## Step 1.5 Send sas_data_full to do S3 (for further analysis using Airflow)

In [447]:
%%time
# Upload data to S3 bucket defined in dwh.cfg in S3 section
upload_files_to_s3('sas_data_full')

CPU times: user 4.21 s, sys: 2.29 s, total: 6.5 s
Wall time: 1min 17s


#### Optinal Analysis
During data exploratio I was wondering how to use US Cities Demographics data.
Bellow is part of my study. But finaly (bacouse of time) I decided to not enriched
Address dimension form I94 SAS. `df_i94addrl` DataFram

In [ ]:
# read SAS data addres

In [32]:
df_cities = pd.read_csv('us-cities-demographics.csv',sep=';')

In [48]:
# define way of aggregation
aggr_meth = {
    'State Code':'count',
    'State':'first',
    'Median Age':'median',
    'Male Population':'sum',
    'Female Population':'sum',
    'Total Population':'sum',
    'Number of Veterans':'sum',
    'Average Household Size':'mean'
}
# column renaming
col_renaming = {
    'State Code':'no_of_cities',
    'State':'state',
    'Median Age':'median_age',
    'Male Population':'male_population',
    'Female Population':'female_population',
    'Total Population':'total_population',
    'Number of Veterans':'no_of_veterans',
    'Average Household Size':'avg_household_size'
}
df_states = df_cities.groupby('State Code').agg(aggr_meth).rename(columns=col_renaming).reset_index()
df_states.head()

,State Code,no_of_cities,state,median_age,male_population,female_population,total_population,no_of_veterans,avg_household_size
0,AK,5,Alaska,32.2,764725.0,728750.0,1493475,137460.0,2.770000
1,AL,34,Alabama,38.0,2448200.0,2715106.0,5163306,352896.0,2.430000
2,AR,29,Arkansas,32.6,1400724.0,1482165.0,2882889,154390.0,2.526897
3,AZ,80,Arizona,34.1,11137275.0,11360435.0,22497710,1322525.0,2.774375
4,CA,676,California,35.8,61055672.0,62388681.0,123444353,4617022.0,3.095325


In [49]:
# get address data from SAS lables Description
df_i94addrl = pd.read_csv('lookup_data/_i94addrl.csv',sep=";")

In [61]:
df_i94addrl_new = pd.merge(df_i94addrl, df_states, left_on='address_code', right_on='State Code')
del df_i94addrl_new['State Code']

In [63]:
df_i94addrl_new.head()

,address_code,address_name,no_of_cities,state,median_age,male_population,female_population,total_population,no_of_veterans,avg_household_size
0,AL,ALABAMA,34,Alabama,38.0,2448200.0,2715106.0,5163306,352896.0,2.430000
1,AK,ALASKA,5,Alaska,32.2,764725.0,728750.0,1493475,137460.0,2.770000
2,AZ,ARIZONA,80,Arizona,34.1,11137275.0,11360435.0,22497710,1322525.0,2.774375
3,AR,ARKANSAS,29,Arkansas,32.6,1400724.0,1482165.0,2882889,154390.0,2.526897
4,CA,CALIFORNIA,676,California,35.8,61055672.0,62388681.0,123444353,4617022.0,3.095325


In [64]:
df_i94addrl.head()

,address_code,address_name
0,AL,ALABAMA
1,AK,ALASKA
2,AZ,ARIZONA
3,AR,ARKANSAS
4,CA,CALIFORNIA


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Base on information I get during Data Exploration I decided to use AWS Redshift DB and Start Schema architecture with staging tables.
I decided to use following schema architecure
![airflow_subdag](./img/db_schema.png)

#### 3.2 Mapping Out Data Pipelines
Target ETL processing was implemented in Airflow using following mapping
![airflow_subdag](./img/file2tbl_mapping.png)

Please see `README.md` file (section **Airflow**) for more details regarding ETL process.

---

## 2. Airflow
### 2.1 How to prepare and run this subproject
For this task I propose to use DWH implementation based on AWS Redshift.
It allows further access for many users (100+) and nice data exploration.
Together with Airflow it is good solution for countiouse ETL pipeline proces.   

### 2.2 AWS Redshift
1. First, edit `dwh.cfg` file. Fill in AWS acces key (KEY) and secret (SECRET).
Rest of fields was provided as example. It could be change base on any further needs.
2. Prepare AWS Environment with Redshift and proper IAM role to access S3 bucket.
- to create AWS Redshift Cluster please run `python aws_env.py -c`
- if you want to delete AWS Redshift Cluster please run `python aws_env.py -d`

### 2.3 Airflow run
In case of this project I decided to use Airflow Project Workspace provided
by Udacity. To run this project (after moving project files to Project Workspace)
please run following command:`/opt/airflow/start.sh`  
![airflow_run](./img/airflow_run.png)

### 2.4 Airflow UI
1. Prepare Airflow's UI to configure your AWS credentials and connection to Redshift.
   * You can use the Project Workspace (from Udacity) and click on the blue Access Airflow button in the bottom right.
   * If you'd prefer to run Airflow locally, open http://localhost:8080 in Google Chrome (other browsers occasionally have issues rendering the Airflow UI). 
2. Create proper Connection in Admin tab
   ![airflow_admin](./img/airflow_admin.png)
   
3. On the create connection page, enter the following values:
- Conn Id: Enter aws_credentials.
- Conn Type: Enter Amazon Web Services.
- Login: Enter your Access key ID from the IAM User credentials.
- Password: Enter your Secret access key from the IAM User credentials.
![aws_credentials](https://video.udacity-data.com/topher/2019/February/5c5aaefe_connection-aws-credentials/connection-aws-credentials.png)  
Once you've entered these values, select Save and Add Another.
  
4. On the next create connection page, enter the following values:
- Conn Id: Enter redshift.  
- Conn Type: Enter Postgres.  
- Host: Enter the endpoint of your Redshift cluster, excluding the port at the end.  
- Schema: Enter dev. This is the Redshift database you want to connect to.  
- Login: Enter awsuser.  
- Password: Enter the password you created when launching your Redshift cluster.  
- Port: Enter 5439.  
![redshift](./img/airflow_connection.png)  
Once you've entered these values, select Save.

### 2.5 AMVisitors DAGs  
For the purpose of this project I have prepared two DAGs:  

![amv_dags](./img/amvisitors_dags.png)  
- **amvisitors_dbschema_dag** - DAG (without schedule) for DB schema initializatin and lookup dimensions Tables filling  
  ![amv_dags](./img/DAG_db_schema_creation.png)  
  This DAG should by start manualy, each time when there is a need to clean environment
  and DB dimension 'lookup table' recreation 
- **amvisitors_etl_dag** - DAG for processing i94 Imigration data events data @mounthly  
  ![amv_dags](./img/DAG_etl.png)  
  To start the DAG please switch state from OFF to ON.
  After Refresing in Tree View of *amvisitors_etl_dag* status of catching up should be visible.  
  ![airflow_dag_tree](./img/DAG_tree.png)  


### 2.6 Data Quality Checks  

Simple Quality Checks were implemented as part of DAGs in Airflow pipeline on two stages:
- after DB Schema creation and Lookup data load
- each time DAG is run every month (including catchup process) simple count of adding records are checked.  

As example of correct values check after Lookup data load I added two tasks:
``` python
run_quality_check_dim_port = DataQualityOperator(
    task_id='Data_quality_check_dim_port',
    dag=dag,
    redshift_conn_id="redshift",
    test_sql = "select lon,lat from dim_port where port_code = 'HOM'",
    test_result = (-151.477,59.6456,)
)


run_quality_check_dim_address = DataQualityOperator(
    task_id='Data_quality_check_dim_address',
    dag=dag,
    redshift_conn_id="redshift",
    test_sql = "select address_name from dim_address where address_code = 'FL'",
    test_result = ('FLORIDA',)
)

```

In production system Quality Check shall be more sophisticated.  

### 2.7 Main files
The project template includes four files:

- `dwh.cfg` main configuration file
- `aws_env.py` is where AWSEnv class was implemented to simplify AWS Redshift Cluster creation
```  
└───airflow                      # Airflow home
|   |               
│   └───dags                     
│   |   │ amvisitors_dbschema_dag.py      # DAG for DB schema creation and lookup table filling
|   |   | amvisitors_etl_dag.py           # Main DAG definition
|   └───plugins
│       │  
|       └───helpers
|       |   | sql_queries.py     # All sql queries needed for AMVisitorsDB creation and ETL process
|       |
|       └───operators
|       |   | data_quality.py    # DataQualityOperator
|       |   | load_dimension.py  # LoadDimensionOperator
|       |   | load_fact.py       # LoadFactOperator
|       |   | load_lookup.py     # LoadLookupToRedshiftOperator
|       |   | stage_redshift.py  # StageToRedshiftOperator
```

## 3. Database Schema (result of Step 3)
Based on Data Exploration and Identification in `Capstone Project TemplateFinal.ipynb` 
I have proposed following Start DB Schema.

![airflow_subdag](./img/db_schema.png)

Why did you choose the model you chose?
Benefits of Star Schema:
* Star schemas are easy for end users and applications to understand and navigate.
* Query performance
* Load performance and administration
* Built-in referential integrity
* Easily understood:

REMARKS:  
1. Becasue and nature of static SAS data description I decided to assume that following
   dimensions tables \[`dim_port`,`dim_mode`,`dim_location`,`dim_address`,`dim_visa`\]
   will be filled ones and during normal ETL process will not be updated.
   
2. Because of AWS RedShift does not enforce constraints [link](https://docs.aws.amazon.com/redshift/latest/dg/r_CREATE_TABLE_NEW.html)
   there was need to use additional `WHERE <pk_column> NOT IN` clause in `sql_queries.py` for
   [`dim_visitor_insert`,`dim_arrdate_time_insert`,`dim_depdate_time_insert`] SQL's.
   


---

## 4. Project Summary and additional topics (Step 5).  
For the purpose of this project I have used:
- Jupyter Environment
- ETL: Python, Pandas, Pyspark
- Data Warehouse: AWS Redshift
- Data Pinelines: Apache Airflow
- Storage: AWS S3

**How often the data should be updated and why?**

I think that nowadays data shall be process as soon as possible, event in near real-time
but it depends on data collection and transformation from third party data sourcess.  

In case of this project and I94 Imigration data - the visitors in each port arrive in and departure at anytime. In production system I think data could be collected with day or even minute precision. It would be efficient to run ETL process hourly or daily.

**What IF?:**
- If the data was increased by 100x.  
  If Redshift DB will be not enought or will be to costly I will considert to use any
  sharding DB solution (SQL or event NoSQL) with horizontal scaling.  
  
- If the pipelines were run on a daily basis by 7am.  
  The Airflow scheduler monitors all tasks and all DAGs, and triggers the task instances
  whose dependencies have been met.  
  
- If the database needed to be accessed by 100+ people.  
  If we use AWS Redshift there is possible to increase any quatas and limites.  
  Plese take a look on [this](https://docs.aws.amazon.com/redshift/latest/mgmt/amazon-redshift-limits.html) 
